In [33]:
import geopandas as gpd
import os

In [34]:
# Set wd
os.chdir('/Users/benediktkorbach/Documents/GitHub/remote-sensing-of-parking-areas')

print("Working directory:", os.getcwd())

Working directory: /Users/benediktkorbach/Documents/GitHub/remote-sensing-of-parking-areas


In [35]:
# Load the GeoJSON files of verified rest stops
parking_areas_ver = gpd.read_file("02_data_acquisition/verified_parking_data/parking_areas_ver.geojson") # polygons of verified car and truck parking space
rest_stations_ver_bbox = gpd.read_file("02_data_acquisition/verified_parking_data/rest_stations_ver_bbox.geojson") # polygons of verified rest stops

In [36]:
rest_stations_ver_bbox

,id_rest,id_OSM_rest,name,geometry
0,lon_8.6630029_lat_50.2542348,way/22568867,Schäferborn,"POLYGON ((8.66300 50.25423, 8.66302 50.25237, ..."
1,lon_11.3531491_lat_50.9378362,way/27549694,Habichtsfang,"POLYGON ((11.35315 50.93784, 11.35315 50.93783..."
2,lon_9.6300119_lat_53.8303266,way/31128689,Steinburg,"POLYGON ((9.63001 53.83033, 9.63273 53.82905, ..."
3,lon_9.6297338_lat_53.8301666,way/31128716,Steinburg,"POLYGON ((9.62973 53.83017, 9.62965 53.83010, ..."
4,lon_11.0102562_lat_50.8353109,way/32489728,Dornheimer Rieth,"POLYGON ((11.01026 50.83531, 11.01145 50.83591..."
...,...,...,...,...
262,lon_7.2526746_lat_52.9419091,way/839676611,Walchum West,"POLYGON ((7.25267 52.94191, 7.25199 52.94181, ..."
263,lon_7.2005154_lat_52.2816225,way/988014767,Gut Friedrichstal,"POLYGON ((7.20052 52.28162, 7.20053 52.28167, ..."
264,lon_8.5597202_lat_49.5969765,way/992004892,Wildbahn,"POLYGON ((8.55972 49.59698, 8.55958 49.59693, ..."
265,lon_10.7306355_lat_50.559048,way/1171916133,Adlersberg,"POLYGON ((10.73064 50.55905, 10.73067 50.55851..."


In [28]:
# Change bbox column to list
rest_stations_ver_bbox['bbox'] = rest_stations_ver_bbox['bbox'].apply(lambda x: x.split(','))

KeyError: 'bbox'

In [11]:
# Define test index
test_index = 5

test_bbox = rest_stations_ver_bbox.iloc[test_index].loc["bbox"]
test_id_rest = rest_stations_ver_bbox.iloc[test_index].loc["id_rest"]
test_name = rest_stations_ver_bbox.iloc[test_index].loc["name"]

print("Test bounding box:", test_bbox)
print("Test id box:", test_id_rest)
print("Test service station:", test_name)

IndexError: single positional indexer is out-of-bounds

In [ ]:
parking_areas = parking_areas_ver[parking_areas_ver["id_rest"] == test_id_rest]

In [1]:
parking_areas

NameError: name 'parking_areas' is not defined

In [ ]:
car_parking = parking_areas[parking_areas["type"] == "car"]
truck_parking = parking_areas[parking_areas["type"] == "truck"]

In [ ]:
test_polygon = parking_areas_ver[parking_areas_ver["id_rest"] == test_id_rest].iloc[0]

In [ ]:
print(test_polygon)

In [ ]:
def create_mask(geodataframe, bbox, size = 3000):

    # Calculate pixel size
    xres = (bbox[2] - bbox[0]) / size
    yres = (bbox[3] - bbox[1]) / size

    # Create an affine transform for the raster
    transform = rasterio.transform.from_bounds(*bbox, size, size)

    # Create the mask
    geometries = geodataframe.geometry
    mask = geometry_mask(geometries, transform=transform, invert=True, out_shape=(size, size))

    return mask

In [ ]:
big_mask = create_mask(parking_areas, test_bbox)

In [ ]:
# Create big mask

car_mask = create_mask(car_parking, test_bbox)
truck_mask = create_mask(truck_parking, test_bbox)

In [ ]:
print(car_mask)

In [ ]:
plt.imshow(car_mask, cmap='gray')
plt.title('Mask of Polygon Geometries')
plt.show()

In [ ]:
plt.imshow(truck_mask, cmap='gray')
plt.title('Mask of Polygon Geometries')
plt.show()

In [ ]:
def split_image_to_tiles(image_path, mask, tile_width, tile_height, output_folder):
    # Open the image
    with Image.open(image_path) as img:
        image_width, image_height = img.size

        # Convert mask to PIL Image if it's not already
        if isinstance(mask, np.ndarray):
            mask = Image.fromarray(mask.astype(np.uint8) * 255)

        # Calculate the number of tiles in each dimension
        x_tiles = image_width // tile_width
        y_tiles = image_height // tile_height

        # Create output folder if it does not exist
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Generate tiles
        for x in range(x_tiles):
            for y in range(y_tiles):
                # Define the bounding box
                left = x * tile_width
                upper = y * tile_height
                right = (x + 1) * tile_width
                lower = (y + 1) * tile_height
                box = (left, upper, right, lower)

                # Crop the image to the bounding box
                tile = img.crop(box)

                # Crop the mask to the bounding box
                mask_tile = mask.crop(box)

                # Check if the tile contains any "true" values in the mask
                if np.any(mask_tile):
                    # Save the tile
                    tile.save(os.path.join(output_folder, f"tile_{x}_{y}.png"))

In [ ]:
output_folder = f"/content/drive/MyDrive/Master Thesis/01 Data Acquisition/Test Download PNG/{test_id_rest}_{test_name}"

split_image_to_tiles(image_path=file_path_tif, mask=truck_mask, tile_width=500, tile_height=500, output_folder=output_folder)

In [ ]:
split_image_to_tiles(image_path='image.jpg', mask=Image.open(mask_path), tile_width=100, tile_height=100, output_folder='tiles')

# Graveyard

In [ ]:
# Assuming gdf is your GeoDataFrame
gdf = parking_areas

# Define your bounding box [left, bottom, right, top]
bbox = test_bbox

# Calculate pixel size
width = height = 3000
xres = (bbox[2] - bbox[0]) / width
yres = (bbox[3] - bbox[1]) / height

# Create an affine transform for the raster
transform = rasterio.transform.from_bounds(*bbox, width, height)

# Create the mask
geometries = gdf.geometry
mask = geometry_mask(geometries, transform=transform, invert=True, out_shape=(height, width))# Load the satellite image
with rasterio.open(file_path_tif) as src:
    data = src.read()
    if data.shape[0] == 3:
        satellite_image = np.moveaxis(data, 0, -1)
    else:
        raise ValueError("The image does not have 3 bands. Check your image data.")

    # Plot the figure with the desired DPI
    dpi = 600  # 3000 pixels / 10 inches
    fig, ax = plt.subplots(figsize=(5, 5), frameon=False, dpi=dpi)

    # Show the satellite image
    show(src, ax=ax)

    # Reproject the GeoDataFrame if necessary
    if parking_areas.crs != src.crs:
        parking_areas = parking_areas.to_crs(src.crs)

    # Plot the polygons
    parking_areas.plot(ax=ax, facecolor='none', edgecolor='red')

    # Remove axis and layout padding
    ax.axis('off')
    plt.tight_layout(pad=0)

    # Save the figure
    #plt.savefig("/content/drive/MyDrive/output_image.png", bbox_inches='tight', pad_inches=0)

    # Display the plot
    plt.show()

In [ ]:
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

with rasterio.open(file_path_tif) as src:
    data = src.read()
    if data.shape[0] == 3:
        satellite_image = np.moveaxis(data, 0, -1)
    else:
        raise ValueError("The image does not have 3 bands. Check your image data.")

    # Set the figure size and DPI for 3000x3000 pixels output
    fig, ax = plt.subplots(figsize=(10, 10), dpi=300)
    canvas = FigureCanvas(fig)

    # Show the satellite image using imshow
    ax.imshow(satellite_image)

    # Remove axis and layout padding
    ax.axis('off')
    plt.tight_layout(pad=0)

    # Draw the figure in memory
    canvas.draw()

    # Convert the figure to a numpy array
    image_array = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
    image_array = image_array.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    # Now, image_array is your 3000x3000 numpy array of the combined image

In [ ]:
plt.imshow(image_array, cmap='gray')

In [ ]:
# save as png

image = Image.fromarray(np.uint8(image_array))

# Save the image
#image.save("/content/drive/MyDrive/Master Thesis/01 Data Acquisition/pngfromarray.png")

In [ ]:
def split_image_to_tiles(image_path, tile_width, tile_height, output_folder):
    # Open the image
    with Image.open(image_path) as img:
        image_width, image_height = img.size

        # Calculate the number of tiles in each dimension
        x_tiles = image_width // tile_width
        y_tiles = image_height // tile_height

        # Create output folder if it does not exist
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Generate tiles
        for x in range(x_tiles):
            for y in range(y_tiles):
                # Define the bounding box
                left = x * tile_width
                upper = y * tile_height
                right = (x + 1) * tile_width
                lower = (y + 1) * tile_height
                box = (left, upper, right, lower)

                # Crop the image to the bounding box
                tile = img.crop(box)

                # Save the tile
                tile.save(os.path.join(output_folder, f"tile_{x}_{y}.png"))

In [ ]:
# Usage
image_path = file_path_tif
tile_width = 500
tile_height = 500
output_folder = "/content/drive/MyDrive/Master Thesis/01 Data Acquisition/Test Download PNG/Test Download tiles"
split_image_to_tiles(image_path, tile_width, tile_height, output_folder)

In [ ]:
# Assuming gdf is your GeoDataFrame
gdf = parking_areas

# Define your bounding box [left, bottom, right, top]
bbox = test_bbox

# Calculate pixel size
width = height = 3000
xres = (bbox[2] - bbox[0]) / width
yres = (bbox[3] - bbox[1]) / height

# Create an affine transform for the raster
transform = rasterio.transform.from_bounds(*bbox, width, height)

# Create the mask
geometries = gdf.geometry
mask = geometry_mask(geometries, transform=transform, invert=True, out_shape=(height, width))

# The mask is a numpy array that can be used for further processing

plt.imshow(mask, cmap='gray')
plt.title('Mask of Polygon Geometries')
plt.show()

In [ ]:
mask.shape